<a href="https://colab.research.google.com/github/GLeviosa/iris-detection/blob/main/iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 1 - Visão Computacional

Alunos: Giovanni Santos, Henrique Thomé


## Instalando dependências

In [1]:
!pip3 install scikit-image --upgrade

     |████████████████████████████████| 29.2 MB 82 kB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
      Successfully uninstalled scikit-image-0.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
!git clone https://github.com/lucianosilva-github/G6_iris_recognition.git

Cloning into 'G6_iris_recognition'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 31 (delta 3), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [3]:
!git clone https://github.com/lucianosilva-github/images.git

Cloning into 'images'...
remote: Enumerating objects: 1266, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 1266 (delta 1), reused 0 (delta 0), pack-reused 1246
Receiving objects: 100% (1266/1266), 189.65 MiB | 22.99 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Checking out files: 100% (1163/1163), done.


## Imports

In [4]:
import G6_iris_recognition
import cv2
import os
import numpy as np
import shutil

## Definindo diretórios locais



In [6]:
raw_path = './images'
sharp_path = './sharpened_images'
test_path = './test_images'

shutil.rmtree(sharp_path, ignore_errors=True)
shutil.rmtree(test_path, ignore_errors=True)

if not os.path.exists(sharp_path):
    os.mkdir(sharp_path)

if not os.path.exists(test_path):
  os.mkdir(test_path)

In [17]:
def crop_it(img, p1, p2):
  return img[p1[1] : img.shape[0] - p2[1]][p1[0] : img.shape[1] - p2[0]]

## Tratamento das imagens

In [19]:
iris_ids = [f"000{x}" if not x//10 else f"00{x}" for x in range(60)]
iris_ids.remove("0012")
times = 0
for iris in iris_ids:
  times = 0
  if iris == '.ipynb_checkpoints' or iris == '.DS_Store':
    continue
  if not os.path.exists(f'{sharp_path}/{iris}'):
    os.mkdir(f'{sharp_path}/{iris}')
  if not os.path.exists(f'{test_path}/{iris}'):
    os.mkdir(f'{test_path}/{iris}')
  for file in os.listdir(f'{raw_path}/{iris}'):

    img = cv2.cvtColor(cv2.imread(f'{raw_path}/{iris}/{file}'), cv2.COLOR_BGR2GRAY)
    h, w = img.shape

    img_copy = img
    y1, y2 =0, img_copy.shape[0]

    init = (20, 35)
    coords = init

    while y2> img.shape[1]*0.5 and y2*0.7 < abs(y1-y2):

      coords = (init[0], coords[1] + 2)
      coords_init = init

      img_crop = crop_it(img_copy, coords, coords_init)
      img_equalized = cv2.equalizeHist(img_crop)
      img_blur = cv2.medianBlur(img_equalized, 25)
      img_border_detected = cv2.Canny(img_blur, 50, 100)

      bord = np.argwhere(img_border_detected > 0)
      y1,x1 = bord.min(axis=0)
      y2,x2 = bord.max(axis=0)

    blank_image = np.zeros((h, w), np.uint8)
    final_crop = img_equalized[y1 : y2 , x1 : x2]

    blank_image[ y1 : y2, x1 : x2 ] = final_crop

    final_crop = blank_image

    img_equalized = cv2.cvtColor(img_equalized, cv2.COLOR_GRAY2BGR)
    rec = cv2.rectangle(img_equalized, (x1,y1), (x2,y2), (0,0,255), 3, cv2.LINE_AA)

    if times < 8:
      cv2.imwrite(f'{sharp_path}/{iris}/{file}', final_crop)
    else:
      cv2.imwrite(f'{test_path}/{iris}/{file}', final_crop)
    times += 1

## Treinamento

In [11]:
G6_iris_recognition.iris_model_train(sharp_path, "model.pickle")

directory_list ['./sharpened_images/0001', './sharpened_images/0042', './sharpened_images/0045', './sharpened_images/0035', './sharpened_images/0000', './sharpened_images/0021', './sharpened_images/0039', './sharpened_images/0010', './sharpened_images/0059', './sharpened_images/0049', './sharpened_images/0041', './sharpened_images/0034', './sharpened_images/0023', './sharpened_images/0043', './sharpened_images/0051', './sharpened_images/0024', './sharpened_images/0025', './sharpened_images/0030', './sharpened_images/0014', './sharpened_images/0013', './sharpened_images/0027', './sharpened_images/0005', './sharpened_images/0006', './sharpened_images/0037', './sharpened_images/0053', './sharpened_images/0007', './sharpened_images/0022', './sharpened_images/0003', './sharpened_images/0047', './sharpened_images/0040', './sharpened_images/0016', './sharpened_images/0058', './sharpened_images/0033', './sharpened_images/0008', './sharpened_images/0048', './sharpened_images/0046', './sharpened

['0021',
 '0059',
 '0034',
 '0024',
 '0014',
 '0013',
 '0027',
 '0006',
 '0058',
 '0008',
 '0038',
 '0028',
 '0036',
 '0004']

## Calculando eficácia do modelo

In [13]:
hit = 0
count = 0

for iris in os.listdir(test_path):
  if iris == '.ipynb_checkpoints' or iris == '.DS_Store':
    continue
    
  for file in os.listdir(f'{test_path}/{iris}'):
    iris_name = G6_iris_recognition.iris_model_test("model.pickle", f'{test_path}/{iris}/{file}')
    print(f"{count}: {iris_name} == {iris} -> {iris_name == iris}")
    if iris_name == iris:
      hit += 1 
    count += 1

print(f"Acertou {hit} de {count} \n% de acertos: {hit/count}")

rectangle expression1
0: unmatch == 0001 -> False
1: 0034 == 0001 -> False
2: 0027 == 0001 -> False
3: 0058 == 0001 -> False
4: 0058 == 0001 -> False
5: 0027 == 0001 -> False
6: 0058 == 0001 -> False
rectangle expression1
7: unmatch == 0001 -> False
8: 0027 == 0001 -> False
rectangle expression1
9: unmatch == 0001 -> False
10: 0058 == 0001 -> False
11: 0027 == 0001 -> False
12: 0058 == 0042 -> False
13: 0058 == 0042 -> False
14: 0058 == 0042 -> False
15: 0058 == 0042 -> False
16: 0058 == 0042 -> False
17: 0058 == 0042 -> False
18: 0058 == 0042 -> False
19: 0058 == 0042 -> False
20: 0058 == 0042 -> False
locate expression 1 local variable 'north_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
21: unmatch == 0042 -> False
22: 0058 == 0042 -> False
locate expression 1 local variable 'north_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
23: unmatch == 0042 -> False
24: 0058 ==